In [5]:
import os

# Remove old credentials file
netrc_path = os.path.expanduser("~/.netrc")
if os.path.exists(netrc_path):
    os.remove(netrc_path)
    print("Old W&B credentials removed.")
else:
    print("No old W&B credentials found.")


Old W&B credentials removed.


In [1]:
import wandb

# Paste your personal W&B API key here
wandb.login(key="69abb0e0f3b4c2258b3ec130d17483f58ded0bf9", relogin=True)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/kalpanapullagura/.netrc
wandb: Currently logged in as: 142502009 (142502009_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
print("Logged in as:", wandb.api.default_entity)


Logged in as: 142502009_


In [9]:
from datasets import load_dataset
import wandb
from collections import Counter

# Load the dataset
dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)

# Get the NER label names
ner_labels = dataset["train"].features["ner_tags"].feature.names

# Count occurrences of each entity
entity_counter = Counter()
for example in dataset["train"]["ner_tags"]:
    for tag in example:
        entity_name = ner_labels[tag]
        if entity_name != "O":
            entity_counter[entity_name.split("-")[-1]] += 1

# Prepare summary metrics
summary_metrics = {
    "num_train_samples": len(dataset["train"]),
    "num_val_samples": len(dataset["validation"]),
    "num_test_samples": len(dataset["test"]),
}

# Add entity counts to summary metrics
for entity, count in entity_counter.items():
    summary_metrics[f"entity_count_{entity}"] = count

# Print and log metrics to Weights & Biases
print("Summary metrics to log:", summary_metrics)

# Added entity parameter here
wandb.init(project="Q1-weak-supervision-ner")
wandb.summary.update(summary_metrics)
wandb.finish()


Summary metrics to log: {'num_train_samples': 14041, 'num_val_samples': 3250, 'num_test_samples': 3453, 'entity_count_ORG': 10025, 'entity_count_MISC': 4593, 'entity_count_PER': 11128, 'entity_count_LOC': 8297}


entity_count_LOC,8297
entity_count_MISC,4593
entity_count_ORG,10025
entity_count_PER,11128
num_test_samples,3453
num_train_samples,14041
num_val_samples,3250


In [10]:
from datasets import load_dataset
import pandas as pd
import re
import wandb
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis

ABSTAIN = -1
MISC = 0
ORG = 1

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
ner_labels = dataset["train"].features["ner_tags"].feature.names

data_for_df = []
for example in dataset["train"]:
    for token, tag_id in zip(example["tokens"], example["ner_tags"]):
        label_str = ner_labels[tag_id]
        base_label = label_str.split("-")[-1]
        data_for_df.append({"word": token, "ner_label": base_label})

df = pd.DataFrame(data_for_df)

wandb.init(
    project="Q1-weak-supervision-ner",
    name="snorkel-lfs",
    reinit=True
)

assert 'df' in locals(), "Please load your dataset as a pandas DataFrame named 'df'."

label_map = {"MISC": MISC, "ORG": ORG}
df['ner_label_int'] = df['ner_label'].map(label_map).fillna(ABSTAIN).astype(int)

entity_counts = df['ner_label'].value_counts().to_dict()
wandb.log({
    "num_train_samples": len(df),
    **{f"entity_count_{k}": v for k, v in entity_counts.items()}
})

@labeling_function()
def lf_year(x):
    if re.match(r"^(19|20)\d{2}$", x['word']):
        return MISC
    return ABSTAIN

@labeling_function()
def lf_org_suffix(x):
    org_suffixes = ["Inc.", "Corp.", "Ltd."]
    if any(suffix in x['word'] for suffix in org_suffixes):
        return ORG
    return ABSTAIN

lfs = [lf_year, lf_org_suffix]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df)

lf_analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary(Y=df['ner_label_int'].to_numpy())
print(lf_analysis)

accuracy_col = 'Emp. Acc.'
for lf_name in lf_analysis.index:
    wandb.log({
        f"{lf_name}_coverage": lf_analysis.loc[lf_name, 'Coverage'],
        f"{lf_name}_accuracy": lf_analysis.loc[lf_name, accuracy_col]
    })

wandb.log({
    "LF Coverage vs Accuracy": wandb.Table(
        dataframe=lf_analysis.reset_index()
    )
})

wandb.finish()


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


100%|███████████████████████████████| 203621/203621 [00:01<00:00, 155837.61it/s]


               j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
lf_year        0      [0]  0.002667       0.0        0.0        3          1   
lf_org_suffix  1      [1]  0.000108       0.0        0.0       22          0   

               Emp. Acc.  
lf_year         0.005525  
lf_org_suffix   1.000000  


entity_count_LOC,▁
entity_count_MISC,▁
entity_count_O,▁
entity_count_ORG,▁
entity_count_PER,▁
lf_org_suffix_accuracy,▁
lf_org_suffix_coverage,▁
lf_year_accuracy,▁
lf_year_coverage,▁
num_train_samples,▁
entity_count_LOC,8297


In [3]:
import numpy as np
import re
import wandb
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling.model import MajorityLabelVoter
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
ner_labels = dataset["train"].features["ner_tags"].feature.names

data_for_df = []
for example in dataset["train"]:
    for token, tag_id in zip(example["tokens"], example["ner_tags"]):
        label_str = ner_labels[tag_id]
        base_label = label_str.split("-")[-1]
        data_for_df.append({"word": token, "ner_label": base_label})

df = pd.DataFrame(data_for_df)

wandb.init(
    project="Q1-weak-supervision-ner",
    name="snorkel-majority-voter",
    reinit=True
)

ABSTAIN = -1
MISC = 0
ORG = 1

@labeling_function()
def lf_year(x):
    if re.match(r"^(19|20)\d{2}$", x['word']):
        return MISC
    return ABSTAIN

@labeling_function()
def lf_org_suffix(x):
    org_suffixes = ["Inc.", "Corp.", "Ltd."]
    if any(suffix in x['word'] for suffix in org_suffixes):
        return ORG
    return ABSTAIN

lfs = [lf_year, lf_org_suffix]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df)

majority_model = MajorityLabelVoter()
df["aggregated_label"] = majority_model.predict(L=L_train)

num_misc_labels = (df["aggregated_label"] == MISC).sum()
num_org_labels = (df["aggregated_label"] == ORG).sum()
num_abstained_labels = (df["aggregated_label"] == ABSTAIN).sum()

wandb.log({
    "num_samples": len(df),
    "num_misc_labels_aggregated": num_misc_labels,
    "num_org_labels_aggregated": num_org_labels,
    "num_abstained_labels_aggregated": num_abstained_labels,
})

inv_label_map = {v: k for k, v in {"MISC": MISC, "ORG": ORG, "ABSTAIN": ABSTAIN}.items()}
df["aggregated_label_str"] = df["aggregated_label"].map(inv_label_map)

labeled_data = df[df["aggregated_label"] != ABSTAIN]
print("Final Aggregated Labels (showing non-abstained samples):")
print(labeled_data[["word", "aggregated_label_str"]])

wandb.log({
    "Aggregated Labels": wandb.Table(dataframe=labeled_data[["word", "aggregated_label_str"]])
})

wandb.finish()


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


100%|███████████████████████████████| 203621/203621 [00:01<00:00, 148133.87it/s]


Final Aggregated Labels (showing non-abstained samples):
        word aggregated_label_str
549     1966                 MISC
557     1967                 MISC
609     1966                 MISC
611     1969                 MISC
654     1970                 MISC
...      ...                  ...
201666  1996                 MISC
201679  1996                 MISC
201899  Inc.                  ORG
201947  1995                 MISC
201957  1995                 MISC

[565 rows x 2 columns]


num_abstained_labels_aggregated,▁
num_misc_labels_aggregated,▁
num_org_labels_aggregated,▁
num_samples,▁
num_abstained_labels_aggregated,203056
num_misc_labels_aggregated,543
num_org_labels_aggregated,22
num_samples,203621


In [13]:
from datasets import load_dataset
import wandb
from collections import Counter


dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)

ner_labels = dataset["train"].features["ner_tags"].feature.names
entity_counter = Counter()
for example in dataset["train"]["ner_tags"]:
    for tag in example:
        entity_name = ner_labels[tag]
        if entity_name != "O":  
            entity_counter[entity_name.split("-")[-1]] += 1


summary_metrics = {
    "num_train_samples": len(dataset["train"]),
    "num_val_samples": len(dataset["validation"]),
    "num_test_samples": len(dataset["test"]),
}


for entity, count in entity_counter.items():
    summary_metrics[f"entity_count_{entity}"] = count

print("Summary metrics to log:", summary_metrics)

wandb.init(project="Q1-weak-supervision-ner")
wandb.summary.update(summary_metrics)  
wandb.finish()


Summary metrics to log: {'num_train_samples': 14041, 'num_val_samples': 3250, 'num_test_samples': 3453, 'entity_count_ORG': 10025, 'entity_count_MISC': 4593, 'entity_count_PER': 11128, 'entity_count_LOC': 8297}


entity_count_LOC,8297
entity_count_MISC,4593
entity_count_ORG,10025
entity_count_PER,11128
num_test_samples,3453
num_train_samples,14041
num_val_samples,3250


In [14]:
pip install snorkel wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [snorkel]
Note: you may need to restart the kernel to use updated packages.


In [1]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier, LFAnalysis
import pandas as pd
import re
import wandb


In [4]:
# Convert dataset to DataFrame for Snorkel
train_data = dataset['train']
df = pd.DataFrame({
    "word": [word for sent in train_data["tokens"] for word in sent],
    "ner": [label for sent_labels in train_data["ner_tags"] for label in sent_labels]
})

# Map label IDs to names
id2label = {i: name for i, name in enumerate(train_data.features["ner_tags"].feature.names)}
df['ner_label'] = df['ner'].map(id2label)


In [19]:

from datasets import load_dataset
import pandas as pd
import re
import wandb
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis

ABSTAIN = -1

MISC = 0
ORG = 1

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
ner_labels = dataset["train"].features["ner_tags"].feature.names


data_for_df = []
for example in dataset["train"]:
    for token, tag_id in zip(example["tokens"], example["ner_tags"]):
        # Get the string representation of the label (e.g., 'B-PER', 'O')
        label_str = ner_labels[tag_id]
        # Get the base entity type (e.g., 'PER', 'O')
        base_label = label_str.split("-")[-1]
        data_for_df.append({"word": token, "ner_label": base_label})

df = pd.DataFrame(data_for_df)



wandb.init(
    project="Q1-weak-supervision-ner",
    name="snorkel-lfs",
    reinit=True
)


assert 'df' in locals(), "Please load your dataset as a pandas DataFrame named 'df'."


label_map = {"MISC": MISC, "ORG": ORG}

df['ner_label_int'] = df['ner_label'].map(label_map).fillna(ABSTAIN).astype(int)


entity_counts = df['ner_label'].value_counts().to_dict()
wandb.log({
    "num_train_samples": len(df),
    **{f"entity_count_{k}": v for k, v in entity_counts.items()}
})


@labeling_function()
def lf_year(x):
    if re.match(r"^(19|20)\d{2}$", x['word']):
        return MISC 
    return ABSTAIN

@labeling_function()
def lf_org_suffix(x):
    org_suffixes = ["Inc.", "Corp.", "Ltd."]
    if any(suffix in x['word'] for suffix in org_suffixes):
        return ORG 
    return ABSTAIN

lfs = [lf_year, lf_org_suffix]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df)


lf_analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary(Y=df['ner_label_int'].to_numpy())
print(lf_analysis)


accuracy_col = 'Emp. Acc.'

for lf_name in lf_analysis.index:
    wandb.log({
        f"{lf_name}_coverage": lf_analysis.loc[lf_name, 'Coverage'],
        f"{lf_name}_accuracy": lf_analysis.loc[lf_name, accuracy_col]
    })

wandb.log({
    "LF Coverage vs Accuracy": wandb.Table(
        dataframe=lf_analysis.reset_index()
    )
})

wandb.finish()

100%|███████████████████████████████| 203621/203621 [00:01<00:00, 163522.52it/s]


               j Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  \
lf_year        0      [0]  0.002667       0.0        0.0        3          1   
lf_org_suffix  1      [1]  0.000108       0.0        0.0       22          0   

               Emp. Acc.  
lf_year         0.005525  
lf_org_suffix   1.000000  


entity_count_LOC,▁
entity_count_MISC,▁
entity_count_O,▁
entity_count_ORG,▁
entity_count_PER,▁
lf_org_suffix_accuracy,▁
lf_org_suffix_coverage,▁
lf_year_accuracy,▁
lf_year_coverage,▁
num_train_samples,▁
entity_count_LOC,8297


In [22]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import wandb
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling.model import MajorityLabelVoter # Correctly imported

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
ner_labels = dataset["train"].features["ner_tags"].feature.names


data_for_df = []
for example in dataset["train"]:
    for token, tag_id in zip(example["tokens"], example["ner_tags"]):
        label_str = ner_labels[tag_id]
        base_label = label_str.split("-")[-1]
        data_for_df.append({"word": token, "ner_label": base_label})

df = pd.DataFrame(data_for_df)




wandb.init(
    project="Q1-weak-supervision-ner",
    name="snorkel-majority-voter",
    reinit=True
)


ABSTAIN = -1
MISC = 0
ORG = 1


@labeling_function()
def lf_year(x):
    if re.match(r"^(19|20)\d{2}$", x['word']):
        return MISC
    return ABSTAIN

@labeling_function()
def lf_org_suffix(x):
    org_suffixes = ["Inc.", "Corp.", "Ltd."]
    if any(suffix in x['word'] for suffix in org_suffixes):
        return ORG
    return ABSTAIN

lfs = [lf_year, lf_org_suffix]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df)


majority_model = MajorityLabelVoter()
df["aggregated_label"] = majority_model.predict(L=L_train)
# --- END OF CORE LOGIC ---


num_misc_labels = (df["aggregated_label"] == MISC).sum()
num_org_labels = (df["aggregated_label"] == ORG).sum()
num_abstained_labels = (df["aggregated_label"] == ABSTAIN).sum()

wandb.log({
    "num_samples": len(df),
    "num_misc_labels_aggregated": num_misc_labels,
    "num_org_labels_aggregated": num_org_labels,
    "num_abstained_labels_aggregated": num_abstained_labels,
})


inv_label_map = {v: k for k, v in {"MISC": MISC, "ORG": ORG, "ABSTAIN": ABSTAIN}.items()}
df["aggregated_label_str"] = df["aggregated_label"].map(inv_label_map)


labeled_data = df[df["aggregated_label"] != ABSTAIN]
print("Final Aggregated Labels (showing non-abstained samples):")
print(labeled_data[["word", "aggregated_label_str"]])

wandb.log({
    "Aggregated Labels": wandb.Table(dataframe=labeled_data[["word", "aggregated_label_str"]])
})


wandb.finish()

100%|███████████████████████████████| 203621/203621 [00:01<00:00, 143064.90it/s]


Final Aggregated Labels (showing non-abstained samples):
        word aggregated_label_str
549     1966                 MISC
557     1967                 MISC
609     1966                 MISC
611     1969                 MISC
654     1970                 MISC
...      ...                  ...
201666  1996                 MISC
201679  1996                 MISC
201899  Inc.                  ORG
201947  1995                 MISC
201957  1995                 MISC

[565 rows x 2 columns]


num_abstained_labels_aggregated,▁
num_misc_labels_aggregated,▁
num_org_labels_aggregated,▁
num_samples,▁
num_abstained_labels_aggregated,203056
num_misc_labels_aggregated,543
num_org_labels_aggregated,22
num_samples,203621


In [12]:
pip install torch torchvision wandb --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def train_model(model, trainloader, testloader, project_name, run_name, epochs=100):
    wandb.init(project=project_name, name=run_name, reinit=True)
    

    model.to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        running_loss, total, correct = 0.0, 0, 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_acc = 100 * correct / total

        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_acc = 100 * val_correct / val_total

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": running_loss / len(trainloader),
            "train_acc": train_acc,
            "val_acc": val_acc
        })
        print(f"Epoch [{epoch+1}/{epochs}] Run: {run_name} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    wandb.finish()
    return model


transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset_100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset_100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
trainloader_100 = DataLoader(trainset_100, batch_size=128, shuffle=True)
testloader_100 = DataLoader(testset_100, batch_size=128, shuffle=False)

trainset_10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset_10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader_10 = DataLoader(trainset_10, batch_size=128, shuffle=True)
testloader_10 = DataLoader(testset_10, batch_size=128, shuffle=False)


print("--- Starting Experiment A: CIFAR-100 -> CIFAR-10 ---")

model_A = SimpleCNN(num_classes=100)

model_A = train_model(model_A, trainloader_100, testloader_100, "CIFAR-Order-Experiment", "A: Train on CIFAR-100")

model_A.fc2 = nn.Linear(model_A.fc2.in_features, 10)

model_A = train_model(model_A, trainloader_10, testloader_10, "CIFAR-Order-Experiment", "A: Transfer to CIFAR-10")


print("\n--- Starting Experiment B: CIFAR-10 -> CIFAR-100 ---")

model_B = SimpleCNN(num_classes=10)

model_B = train_model(model_B, trainloader_10, testloader_10, "CIFAR-Order-Experiment", "B: Train on CIFAR-10")

model_B.fc2 = nn.Linear(model_B.fc2.in_features, 100)

model_B = train_model(model_B, trainloader_100, testloader_100, "CIFAR-Order-Experiment", "B: Transfer to CIFAR-100")

--- Starting Experiment A: CIFAR-100 -> CIFAR-10 ---


Epoch [1/100] Run: A: Train on CIFAR-100 | Train Acc: 13.02% | Val Acc: 19.83%
Epoch [2/100] Run: A: Train on CIFAR-100 | Train Acc: 23.45% | Val Acc: 25.70%
Epoch [3/100] Run: A: Train on CIFAR-100 | Train Acc: 28.66% | Val Acc: 29.67%
Epoch [4/100] Run: A: Train on CIFAR-100 | Train Acc: 32.42% | Val Acc: 31.98%
Epoch [5/100] Run: A: Train on CIFAR-100 | Train Acc: 35.44% | Val Acc: 35.74%
Epoch [6/100] Run: A: Train on CIFAR-100 | Train Acc: 37.78% | Val Acc: 35.88%
Epoch [7/100] Run: A: Train on CIFAR-100 | Train Acc: 40.03% | Val Acc: 38.73%
Epoch [8/100] Run: A: Train on CIFAR-100 | Train Acc: 41.62% | Val Acc: 38.53%
Epoch [9/100] Run: A: Train on CIFAR-100 | Train Acc: 42.97% | Val Acc: 40.11%
Epoch [10/100] Run: A: Train on CIFAR-100 | Train Acc: 44.63% | Val Acc: 40.78%
Epoch [11/100] Run: A: Train on CIFAR-100 | Train Acc: 45.52% | Val Acc: 40.43%
Epoch [12/100] Run: A: Train on CIFAR-100 | Train Acc: 46.76% | Val Acc: 42.00%
Epoch [13/100] Run: A: Train on CIFAR-100 | Train

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█
train_acc,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
train_loss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▄▅▅▆▆▆▇▇▇▇▇▇██████████████████████████
epoch,100
train_acc,66.342
train_loss,1.15503
val_acc,46.15


Epoch [1/100] Run: A: Transfer to CIFAR-10 | Train Acc: 61.29% | Val Acc: 67.59%
Epoch [2/100] Run: A: Transfer to CIFAR-10 | Train Acc: 69.70% | Val Acc: 69.07%
Epoch [3/100] Run: A: Transfer to CIFAR-10 | Train Acc: 71.94% | Val Acc: 72.01%
Epoch [4/100] Run: A: Transfer to CIFAR-10 | Train Acc: 73.76% | Val Acc: 72.66%
Epoch [5/100] Run: A: Transfer to CIFAR-10 | Train Acc: 74.78% | Val Acc: 73.98%
Epoch [6/100] Run: A: Transfer to CIFAR-10 | Train Acc: 75.80% | Val Acc: 73.35%
Epoch [7/100] Run: A: Transfer to CIFAR-10 | Train Acc: 76.73% | Val Acc: 73.50%
Epoch [8/100] Run: A: Transfer to CIFAR-10 | Train Acc: 77.04% | Val Acc: 75.38%
Epoch [9/100] Run: A: Transfer to CIFAR-10 | Train Acc: 78.01% | Val Acc: 74.46%
Epoch [10/100] Run: A: Transfer to CIFAR-10 | Train Acc: 78.03% | Val Acc: 75.53%
Epoch [11/100] Run: A: Transfer to CIFAR-10 | Train Acc: 78.86% | Val Acc: 75.43%
Epoch [12/100] Run: A: Transfer to CIFAR-10 | Train Acc: 79.17% | Val Acc: 75.23%
Epoch [13/100] Run: A: Tr

epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇████
train_acc,▁▃▄▄▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
train_loss,█▇▇▆▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▄▄▄▅▆▆▇▆▇▇▇▇▇▇▇▇▇██▇██▇███████████████
epoch,100
train_acc,90.07
train_loss,0.28009
val_acc,78.29



--- Starting Experiment B: CIFAR-10 -> CIFAR-100 ---


Epoch [1/100] Run: B: Train on CIFAR-10 | Train Acc: 43.61% | Val Acc: 52.09%
Epoch [2/100] Run: B: Train on CIFAR-10 | Train Acc: 57.09% | Val Acc: 60.35%
Epoch [3/100] Run: B: Train on CIFAR-10 | Train Acc: 62.88% | Val Acc: 60.92%
Epoch [4/100] Run: B: Train on CIFAR-10 | Train Acc: 66.16% | Val Acc: 66.60%
Epoch [5/100] Run: B: Train on CIFAR-10 | Train Acc: 68.40% | Val Acc: 68.60%
Epoch [6/100] Run: B: Train on CIFAR-10 | Train Acc: 70.33% | Val Acc: 69.81%
Epoch [7/100] Run: B: Train on CIFAR-10 | Train Acc: 71.55% | Val Acc: 70.59%
Epoch [8/100] Run: B: Train on CIFAR-10 | Train Acc: 72.96% | Val Acc: 71.82%
Epoch [9/100] Run: B: Train on CIFAR-10 | Train Acc: 74.13% | Val Acc: 72.19%
Epoch [10/100] Run: B: Train on CIFAR-10 | Train Acc: 74.80% | Val Acc: 73.83%
Epoch [11/100] Run: B: Train on CIFAR-10 | Train Acc: 75.86% | Val Acc: 73.89%
Epoch [12/100] Run: B: Train on CIFAR-10 | Train Acc: 76.74% | Val Acc: 74.35%
Epoch [13/100] Run: B: Train on CIFAR-10 | Train Acc: 77.41% 

epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train_acc,▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
train_loss,█▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▃▄▄▆▆▆▇▇▇▇▇▇▇█▇██▇█▇██████████████████
epoch,100
train_acc,91.212
train_loss,0.24948
val_acc,79.84


Epoch [1/100] Run: B: Transfer to CIFAR-100 | Train Acc: 26.27% | Val Acc: 34.34%
Epoch [2/100] Run: B: Transfer to CIFAR-100 | Train Acc: 37.94% | Val Acc: 38.49%
Epoch [3/100] Run: B: Transfer to CIFAR-100 | Train Acc: 41.44% | Val Acc: 41.69%
Epoch [4/100] Run: B: Transfer to CIFAR-100 | Train Acc: 43.89% | Val Acc: 41.89%
Epoch [5/100] Run: B: Transfer to CIFAR-100 | Train Acc: 45.94% | Val Acc: 43.18%
Epoch [6/100] Run: B: Transfer to CIFAR-100 | Train Acc: 47.25% | Val Acc: 44.29%
Epoch [7/100] Run: B: Transfer to CIFAR-100 | Train Acc: 48.46% | Val Acc: 45.02%
Epoch [8/100] Run: B: Transfer to CIFAR-100 | Train Acc: 49.55% | Val Acc: 45.76%
Epoch [9/100] Run: B: Transfer to CIFAR-100 | Train Acc: 50.80% | Val Acc: 45.78%
Epoch [10/100] Run: B: Transfer to CIFAR-100 | Train Acc: 51.73% | Val Acc: 46.98%
Epoch [11/100] Run: B: Transfer to CIFAR-100 | Train Acc: 52.38% | Val Acc: 46.83%
Epoch [12/100] Run: B: Transfer to CIFAR-100 | Train Acc: 53.07% | Val Acc: 47.15%
Epoch [13/100

epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇███
train_acc,▁▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██████████
train_loss,█▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▆▆▆▆▇▇▇▇▇█▇▇▇█▇█▇█▇██▇██▇▇▇▇█▇▇███▇▇▇█
epoch,100
train_acc,71.69
train_loss,0.94851
val_acc,50.36
